In [1]:
import torch
import torchvision
import pandas as pd
import numpy as np
import glob
import os
import sys
from sklearn.model_selection import train_test_split
from tqdm import tqdm
tqdm.pandas()
import sys
sys.path.append('../scripts/')
from torch_bad_ones import find_bad_ones_torch
from torch.utils.data import Dataset, DataLoader
import PIL
import collections
import math
from torch.nn import Module, Sequential
from torch.optim import Adam
from torchsummary import summary
from torch.autograd import Variable
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
train_files = glob.glob('../../ml-data-training/ship_segmentation_data/train_v2/*.jpg')

In [3]:
print(len(train_files))

192556


In [4]:
train_csv = pd.read_csv('../../ml-data-training/ship_segmentation_data/train_ship_segmentations_v2.csv')

In [5]:
train_csv.head()

,ImageId,EncodedPixels
0,00003e153.jpg,NaN
1,0001124c7.jpg,NaN
2,000155de5.jpg,264661 17 265429 33 266197 33 266965 33 267733...
3,000194a2d.jpg,360486 1 361252 4 362019 5 362785 8 363552 10 ...
4,000194a2d.jpg,51834 9 52602 9 53370 9 54138 9 54906 9 55674 ...


In [6]:
updated_train_csv = train_csv.groupby('ImageId')['EncodedPixels'].apply(list).reset_index()

In [7]:
def create_classification_labels(data):
    try:
        if math.isnan(data[0]) == True:
            return 0
    except:
        return 1
    # else:
    #     return 1

In [8]:
updated_train_csv['class_labels'] = updated_train_csv['EncodedPixels'].progress_apply(create_classification_labels)

100%|██████████| 192556/192556 [00:00<00:00, 289314.26it/s]


In [9]:
updated_train_csv

,ImageId,EncodedPixels,class_labels
0,00003e153.jpg,[nan],0
1,0001124c7.jpg,[nan],0
2,000155de5.jpg,[264661 17 265429 33 266197 33 266965 33 26773...,1
3,000194a2d.jpg,[360486 1 361252 4 362019 5 362785 8 363552 10...,1
4,0001b1832.jpg,[nan],0
...,...,...,...
192551,fffedbb6b.jpg,[nan],0
192552,ffff2aa57.jpg,[nan],0
192553,ffff6e525.jpg,[nan],0
192554,ffffc50b4.jpg,[nan],0


In [10]:
updated_train_csv.head()

,ImageId,EncodedPixels,class_labels
0,00003e153.jpg,[nan],0
1,0001124c7.jpg,[nan],0
2,000155de5.jpg,[264661 17 265429 33 266197 33 266965 33 26773...,1
3,000194a2d.jpg,[360486 1 361252 4 362019 5 362785 8 363552 10...,1
4,0001b1832.jpg,[nan],0


In [11]:
updated_train_csv['fixed_paths'] = updated_train_csv['ImageId'].progress_apply(lambda x: f"../../ml-data-training/ship_segmentation_data/train_v2/" + x)

100%|██████████| 192556/192556 [00:00<00:00, 310742.14it/s]


In [12]:
updated_train_csv.head()

,ImageId,EncodedPixels,class_labels,fixed_paths
0,00003e153.jpg,[nan],0,../../ml-data-training/ship_segmentation_data/...
1,0001124c7.jpg,[nan],0,../../ml-data-training/ship_segmentation_data/...
2,000155de5.jpg,[264661 17 265429 33 266197 33 266965 33 26773...,1,../../ml-data-training/ship_segmentation_data/...
3,000194a2d.jpg,[360486 1 361252 4 362019 5 362785 8 363552 10...,1,../../ml-data-training/ship_segmentation_data/...
4,0001b1832.jpg,[nan],0,../../ml-data-training/ship_segmentation_data/...


In [13]:
for x in updated_train_csv['fixed_paths'].values.tolist():
    if os.path.exists(x) == False:
        print(x)

In [14]:
to_check = updated_train_csv['fixed_paths'].values.tolist()

In [15]:
issues = find_bad_ones_torch(to_check)

In [16]:
updated_train_csv = updated_train_csv[updated_train_csv.fixed_paths != issues[0]]

In [17]:
def split_datasets(data, test_size = 0.01):
    train, test = train_test_split(data, test_size = test_size, random_state = 42) 
    train, val = train_test_split(train, test_size = test_size, random_state = 42)
    return train, val, test

In [18]:
train, val, test = split_datasets(updated_train_csv)

In [19]:
def read_images_make_numpy(csv_file):
    img_list = csv_file['fixed_paths'].values.tolist()
    labels = csv_file['class_labels'].values.tolist()
    img_arr = np.zeros(shape=((32, 3, 256, 256)))
    label_arr = np.zeros(shape=(32, 1))
    for x in tqdm(range(32)):
        img = PIL.Image.open(img_list[x])
        img = img.resize((256, 256))
        img = np.array(img)
        img = img / 255
        img = img.reshape((3, 256, 256))
        img_arr[x] = img
        label_arr[x] = labels[x]
    return (img_arr, label_arr)

In [20]:
train_data = read_images_make_numpy(train)

100%|██████████| 32/32 [00:00<00:00, 82.61it/s]


In [21]:
train_data_XXX = train_data[0].copy()
train_data_YYY = train_data[1].copy()

In [22]:
train_x = torch.Tensor(train_data_XXX).float()
train_y = torch.Tensor(train_data_YYY).float()

In [25]:
train_x = train_x.cuda()
train_y = train_y.cuda()

In [41]:
class Net(Module):
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            torch.nn.Conv2d(3, 4, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(4),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(4),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Flatten(),
        )

        self.linear_layers = Sequential(
            torch.nn.Linear(16384, 1),
            torch.nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.cnn_layers(x)
        # You can use flatten or this below line....
        # x = x.view(x.size(0), -1)
        x = self.linear_layers(x) 
        return x

In [33]:
model = Net()
optimizer = Adam(model.parameters(), lr=0.07)
criterion = torch.nn.BCELoss()
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
# print(model)

In [36]:
def train(epoch):
    model.train()
    tr_loss = 0
    # getting the training set
    # x_train, y_train = Variable(train_x), Variable(train_y)
    # getting the validation set
    # converting the data into GP format
    # if torch.cuda.is_available():
    #     x_train = train_x.cuda()
    #     y_train = train_y.cuda()
        # x_val = x_val.cuda()
        # y_val = y_val.cuda()

    # clearing the Gradients of the model parameters
    optimizer.zero_grad()
    
    # prediction for training and validation set
    output_train = model(train_x)

    # # computing the training and validation loss
    loss_train = criterion(output_train, train_y)
    # train_losses.append(loss_train)
    # val_losses.append(loss_val)

    # # computing the updated weights of all the model parameters
    loss_train.backward()
    output_train = model(train_x)
    loss_train_1 = criterion(output_train, train_y)
    # optimizer.step()
    # tr_loss = loss_train.item()
    # if epoch%2 == 0:
    #     # printing the validation loss
    #     print('Epoch : ',epoch+1, '\t', 'loss :', )
    return output_train, loss_train, loss_train_1

In [37]:
o_t, o_l, o_l_1  = train(100)

In [40]:
o_l_1

tensor(0.5963, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)

In [2]:
print(torch.version.cuda)
print(torch.backends.cudnn.version())

11.7
8500


In [3]:
torch.__version__

'2.0.0+cu117'